Sumário
* [RNN](#rnn)
* [Server](#server)
* [Base Station](#base-station) - [Local](#bs-local)
* [User](#user)
* [Dataset](#dataset)
* [Create BSs, Users and Server](#create_bs_us)
* [Predicting](#predicting)
* [Training](#training)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Importação de bibliotecas, definição de hiperparâmetros e classes

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import copy
import math
from scipy.spatial import distance
import matplotlib.pyplot as plt
import torch.nn.functional as F
import csv

# Set the device to use
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Device -> ", device)

# Define the hyperparameters
num_features = 3
input_size = num_features
hidden_size = 66
output_size = num_features
learning_rate = 0.001
rounds = 1000
epochs = 40
torch_seed = 0

num_base_stations = 5   #Number of Base Stations
num_users         = 20  #Number of users

rows_per_partition = 3600  #1 hour (3600 seconds) per user
num_user_regs = 5          #Number of regs (location and orientation) received by round
num_user_regs_per_time = 1 #1 reg with location and orientation by second
accuracy_threshold = 99.5  #Model accuracy threshold

dir_base = '/content/drive/My Drive/SBRC 2024/'

gru_training_model     = 1
esn_training_model     = 2
lstm_training_model    = 3

current_training_model = gru_training_model

fitness_filter = True

<a name="rnn"></a>
Definindo redes neurais recorrente (RNN):
* 3 neurônios na camada de entrada (input)
* 66 neurônios na camada intermediária (hidden)
* 3 neurônios na camada de saída (output)

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden state with zeros
        h = torch.zeros(1, x.shape[0], self.hidden_size).to(device)

        # Forward propagate RNN
        out, h = self.rnn(x, h)

        # Decode hidden state of last time step
        out = self.fc(out[:, -1, :])

        return out

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.num_layers  = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward pass
        out, _ = self.lstm(x, (h0, c0))

        # Use the output at the last time step
        out = self.fc(out[:, -1, :])

        return out

In [ ]:
class EchoStateNetwork(nn.Module):
    def __init__(self, input_size, reservoir_size, output_size, connectivity=0.1):
        super(EchoStateNetwork, self).__init__()
        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.output_size = output_size

        # Define layers using nn.Linear
        self.input = nn.Linear(input_size, reservoir_size, bias=False)
        self.reservoir = nn.Linear(reservoir_size, reservoir_size, bias=False)
        self.output = nn.Linear(reservoir_size, output_size, bias=False)

        # Initialize reservoir weights
        nn.init.xavier_uniform_(self.input.weight)
        nn.init.xavier_uniform_(self.reservoir.weight)
        nn.init.xavier_uniform_(self.output.weight)

        # Sparsify the reservoir weights
        mask = (torch.rand(reservoir_size, reservoir_size) < connectivity).float()
        self.reservoir.weight.data *= mask

    def forward(self, input_data):
      predictions = []
      X = torch.zeros(input_data.size(0), self.reservoir_size, device=input_data.device)  # Reservoir activations

      for inp in input_data.unbind(1):  # Unbind along the time steps
          X = torch.tanh(self.input(inp) + self.reservoir(X))
          prediction = self.output(X)
          predictions.append(prediction)

      return torch.cat(predictions)

<a name="server"></a>
Definindo uma classe que representa o servidor (Server) da federação (Federated Learning)

In [ ]:
class Server():
    def __init__(self, num_users):
      self.users_global_model = {}
      self.users_global_model_accuracy = {}

    def update_global_model(self, base_stations):

      for us_id in range(num_users):

        #Only those models who needs will be be trainned
        if(us_id in self.users_global_model_accuracy and
           self.users_global_model_accuracy[us_id] >= accuracy_threshold):
          continue

        us_models = []
        us_models_accuracy = []

        #Getting the last User's global model (if exists)
        if (us_id in self.users_global_model):
          us_models.append(self.users_global_model[us_id])

        #Getting models of the same User in different BS
        for bs in base_stations:
          us_models.append(bs.users[us_id].model)
          us_models_accuracy.append(bs.users[us_id].model_accuracy)

        #Models with too lower accuracy will not be aggregated
        if(fitness_filter):
          self._gather_models_capable_to_aggregation(us_models_accuracy, us_models)

        #Setting the user's global model through the new global model for one more round
        self.users_global_model[us_id] = self._aggregate_user_global_model(us_id, us_models)

        #Check model's accuracy
        self.users_global_model_accuracy[us_id] = sum(us_models_accuracy) / len(us_models_accuracy)

        #Updating the User in each Base Station with the global model
        for bs in base_stations:
            bs.users[us_id].model = self.users_global_model[us_id]

    def _gather_models_capable_to_aggregation(self, us_models_accuracy, us_models):
      mean_accuracy = np.mean(us_models_accuracy)
      std_accuracy = np.std(us_models_accuracy)

      for i, accuracy in enumerate(us_models_accuracy):
        if( mean_accuracy - accuracy > std_accuracy):
          del us_models[i]
          del us_models_accuracy[i]

    def _aggregate_user_global_model(self, us_id, us_models):
      with torch.no_grad():
        # Initializing a new model with the same architecture and parameter shapes
        new_model = copy.deepcopy(us_models[0])

        if(current_training_model == esn_training_model):
          local_output_layers = []

          # Collect output layer parameters of the local models
          for model in us_models:
              local_output_layers.append(model.output.weight.data.clone())

          # Aggregate only the output layer parameters using FedAVG
          avg_output_params = sum(local_output_layers) / len(us_models)
          new_model.output.weight.data = avg_output_params.clone()
        else:
          # Zero out all of the parameters in new_model
          for param in new_model.parameters():
              param.data *= 0

          # Sum up the parameter tensor values from all models
          for model in us_models:
              for param, new_param in zip(model.parameters(), new_model.parameters()):
                  new_param.data += param.data

          # Compute the average (FedAVG) by dividing by the number of models
          for param in new_model.parameters():
              param.data /= len(us_models)

      # Model updated
      return new_model

    def can_stop_federation(self):
      has_achieved_needed_accuracy = 0

      for accuracy in self.users_global_model_accuracy.values():
        if(accuracy >= accuracy_threshold):
          has_achieved_needed_accuracy += 1

      return True if len(self.users_global_model_accuracy.items()) == has_achieved_needed_accuracy else False

    def print(self):
      for k, v in self.users_global_model_accuracy.items():
        print("User {} model's mean accuracy = {} [{}]".format(k, v, "READY" if v >= accuracy_threshold else "TRAINING"))

<a name="base-station"></a>
Definindo uma classe que representa uma Base Station (BS)

In [ ]:
class BaseStation():
    bs_id = 0

    def __init__(self, position):
      self.id = BaseStation.bs_id

      self.x = position[0]
      self.y = position[1]

      self.users = []
      self.us_id = 0 #Users ID

      BaseStation.bs_id += 1

    def add_user(self):
      self.users.append(User(self.us_id))

      self.us_id += 1

    def update_user_data(self, us_id, regs):
      perturbed_regs = self._add_perturbation(regs)

      self.users[us_id].set_data(perturbed_regs)

    def _add_perturbation(self, regs):
      regs_perturbed = regs.copy()

      for reg in regs_perturbed:
        us_x = reg[0]
        us_y = reg[1]

        #Sigma based on the distance between user and BS
        sigma = 0.1 * distance.euclidean((us_x, us_y), (self.x, self.y))

        np.random.seed(0)
        perturb = math.sqrt(sigma) * np.random.rand(1,3)

        #User's location and orientation in BS's perception
        reg += perturb[0]

      return regs_perturbed

<a name="user"></a>
Definindo uma classe que representa um Usuário (User)

In [ ]:
class User():

  def __init__(self, id):
    self.id = id

    torch.manual_seed(torch_seed)

    self.inputs  = []
    self.targets = []

    self.model_accuracy = 0

    self._set_model()

  def _set_model(self):

    if(current_training_model == esn_training_model):
      self.model = EchoStateNetwork(input_size, hidden_size, output_size).to(device)
    elif(current_training_model == gru_training_model):
      self.model = GRUModel(input_size, hidden_size, output_size).to(device)
    elif(current_training_model == lstm_training_model):
      self.model = LSTMModel(input_size, hidden_size, 1, output_size).to(device)
    else:
      print('Model undefined')
      1/0

  def set_data(self, data):

    self.test_target = data[-1]

    #Removing last reg for accuracy testing target
    data = data[:len(data) - 1]

    self.test_input = data[-1]

    #Removing last reg for accuracy testing input
    data = data[:len(data) - 1]

    #Reshape to match the RNN input shape
    data = data.reshape(-1, num_user_regs_per_time, input_size)

    self.inputs  = data[:-1] #All regs except the last to train
    self.targets = data[1:].squeeze()  #All regs except the first to test

  def train_model(self):

    self.model.train()

    # Define optimizer and loss function
    optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate, weight_decay=1e-7)
    criterion = nn.MSELoss().to(device)

    inputs_tensor = torch.from_numpy(self.inputs).float().to(device)
    targets_tensor = torch.from_numpy(self.targets).float().to(device)

    # Train the RNN model
    for epoch in range(epochs):
        outputs = self.model(inputs_tensor)
        loss = criterion(outputs, targets_tensor)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    self._get_model_accuracy()

  def _get_model_accuracy(self):
    tracking_input = torch.from_numpy(self.test_input)
    tracking_input = tracking_input.float().to(device)
    tracking_input = tracking_input[None, None, :]

    self.model.eval()

    with torch.no_grad():
      tracking_predicted = self.model(tracking_input)

    self.model_accuracy = 100 - self._get_mean_absolute_percentage_error(self.test_target, tracking_predicted[:, :3])

  def _get_mean_absolute_percentage_error(self, target, predicted):
      mape_x = abs((predicted[0][0] - target[0]) / target[0]) * 100
      mape_y = abs((predicted[0][1] - target[1]) / target[1]) * 100
      mape_o = abs((predicted[0][2] - target[2]) / target[2]) * 100

      #overall MAPE value limited by 100
      mape = (mape_x + mape_y + mape_o) / 3
      mape = min(mape.item(), 100)

      return mape

<a name="bs-local"></a>
Definindo (empíricamente) as posições das Base Stations

In [ ]:
base_stations_locations = []

#Empirical positions
base_stations_locations.append([185.07, 656.36])
base_stations_locations.append([265.54, 137.68])
base_stations_locations.append([703.75, 448.22])
base_stations_locations.append([948.82, 829.26])
base_stations_locations.append([1010.93, 322.32])

locations_mean = np.mean(base_stations_locations)
locations_std  = np.std(base_stations_locations)

#Normalized positions
base_stations_locations = (base_stations_locations - locations_mean) / locations_std

<a name="dataset"></a>
Carregando arquivo CSV em um dataset

O dataset será subdividido em 20 partes iguais para que seja possível distribuir dados de cada um dos 20 usuários presentes no dataset entre as 5 base stations

In [ ]:
# Load the CSV file into a Pandas DataFrame
caminho_arquivo = dir_base + "20_users_1_hour_sorted.csv"
dataset = pd.read_csv(caminho_arquivo, header=0, usecols=["x", "y", "o"])

dataset = dataset.to_numpy()

partitions = []

# Split the DataFrame into partitions
for i in range(num_users):
    s_row = i * rows_per_partition
    e_row = s_row + rows_per_partition
    partitions.append(dataset[s_row:e_row])

partitions_mean = np.mean(partitions)
partitions_std = np.std(partitions)

# Normalized data
partitions = (partitions - partitions_mean) / partitions_std

<a name="create_bs_us"></a>
Criando Base Stations e Users and the Server

In [ ]:
def create_scenario():
  BaseStation.bs_id = 0

  base_stations = []

  for i in range(num_base_stations):
    base_station = BaseStation(base_stations_locations[i])

    for i in range(num_users):
      base_station.add_user()

    base_stations.append(base_station)

  server = Server(num_users)

  sModel = None

  if(current_training_model == esn_training_model):
    sModel = "ESN RNN"
  elif(current_training_model == gru_training_model):
    sModel = "GRU RNN"
  elif(current_training_model == lstm_training_model):
    sModel = "LSTM RNN"

  print("Current training model: " + sModel)

  return base_stations, server

<a name="predicting"></a>
Realizando predições...

In [ ]:
def getting_predictions(server, base_stations):
  #Penultimate (input) and last (target) seconds of each 10 minutes of walking not used in training
  plot_row = 3058

  for us_id, us_model in server.users_global_model.items():
    for bs in base_stations:

      user = bs.users[us_id]

      tracking_input = torch.from_numpy(partitions[us_id][plot_row: plot_row + 1])
      tracking_input = tracking_input.float().to(device)
      tracking_input = tracking_input[None, :]

      #Prediciton on global (from server) model
      us_model.eval()
      with torch.no_grad():
        prediction = us_model(tracking_input).cpu().data.numpy()

      #Denormalizing data
      prediction = (prediction * partitions_std) + partitions_mean
      target = (partitions[us_id][plot_row + 1: plot_row + 2] * partitions_std) + partitions_mean

      a = (target[0][0],  target[0][1])
      b = (prediction[0][0], prediction[0][1])

      euclidean_distance = distance.euclidean(a, b)

      input = (partitions[us_id][plot_row: plot_row + 1] * partitions_std) + partitions_mean

      if(us_id == 0):
        print("Base Station {}".format(str(bs.id)))
        print("User", us_id, "input", (input[0][0], input[0][1], input[0][2]))
        print("User", us_id, "target", (target[0][0], target[0][1], target[0][2]))
        print("User", us_id, "predicted", (prediction[0][0], prediction[0][1], prediction[0][2]))
        print("The Euclidean distance for user", us_id, "=", euclidean_distance, "\n")

In [ ]:
def save_user_data(user_id, data):
  csv_file_sufix = None
  pth_file_sufix = None

  if(current_training_model == esn_training_model):
    csv_file_sufix = "_esn_model_tracking_data.csv"
    pth_file_sufix = "_esn_model.pth"
  elif(current_training_model == gru_training_model):
    csv_file_sufix = "_gru_model_tracking_data.csv"
    pth_file_sufix = "_gru_model.pth"
  elif(current_training_model == lstm_training_model):
    csv_file_sufix = "_lstm_model_tracking_data.csv"
    pth_file_sufix = "_lstm_model.pth"

  csv_tracking_file = dir_base + ("results_sbrc_models_fl-cfa/" if fitness_filter else "results_sbrc_models_fl-sfa/") + str(user_id) + csv_file_sufix

  with open(csv_tracking_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

  model_path = dir_base + ("results_sbrc_models_fl-cfa/" if fitness_filter else "results_sbrc_models_fl-sfa/") + "user_" + str(user_id) + pth_file_sufix

  torch.save(server.users_global_model[user_id].state_dict(), model_path)

In [ ]:
def prepare_predictions_for_save(server):

  for user_id in range(0, 20):
    user_id = user_id

    #Penultimate (input) and last (target) seconds of each 10 minutes of walking not used in training
    plot_row = 3058

    user_tracking_data = []

    #Ten minutes
    for i in range(1,11):
        input = torch.from_numpy(partitions[user_id][plot_row: plot_row + 1])
        input = input.float().to(device)
        input = input[None, :]

        server.users_global_model[user_id].eval()
        with torch.no_grad():
          prediction = server.users_global_model[user_id](input).cpu().data.numpy()

        #Denormalizing data
        prediction = (prediction * partitions_std) + partitions_mean
        target = (partitions[user_id][plot_row + 1: plot_row + 2] * partitions_std) + partitions_mean

        input =  (input.cpu().data.numpy() * partitions_std) + partitions_mean
        user_tracking_data.append([input, target, prediction])

        plot_row += 60 #plus 1 minute

    save_user_data(user_id, user_tracking_data)

  #Input data have to be normalized before prediction (using the models)
  params_path = dir_base + ("results_sbrc_models_fl-cfa/" if fitness_filter else "results_sbrc_models_fl-sfa/") + "normalize_params.pth"

  torch.save({'mean': partitions_mean, 'std': partitions_std}, params_path)

<a name="training"></a>
A cada *round* uma quantidade de registros da caminhada do User (localização, orientação) serão adicionados a cada uma das Base Station (BS). O objetivo é simular o aumento de registros em cada BS ao longo do tempo (*rounds*).
Foi definido que a cada *round* cada BS será incrementada com 5 segundos (5 registros de caminhada) de cada User.

In [ ]:
columns = ['Rodada', 'ID_Usuario', 'Acuracia', 'Status']

def gather_accuracy_per_round(df, server):
      for k, v in server.users_global_model_accuracy.items():
        status = "READY" if v >= accuracy_threshold else "TRAINING"

        temp_df = pd.DataFrame({
            'Rodada': [round],
            'ID_Usuario': [k],
            'Acuracia': [v],
            'Status': [status]
        })

        df = pd.concat([df, temp_df], ignore_index=True)

def save_accuracy_per_round(df):
  csv_accuracy_per_round = dir_base + ("results_sbrc_models_fl-cfa/" if fitness_filter else "results_sbrc_models_fl-sfa/") + "csv_accuracy_per_round"

  if(current_training_model == gru_training_model):
    csv_accuracy_per_round += "_gru.csv"
  elif(current_training_model == esn_training_model):
    csv_accuracy_per_round += "_esn.csv"
  else:
    csv_accuracy_per_round += "_lstm.csv"

  df.to_csv(csv_accuracy_per_round, index=False)

for current_training in range(1,4):
  df = pd.DataFrame(columns=columns)

  current_training_model = current_training

  base_stations, server = create_scenario()

  e_row = num_user_regs

  for round in range(rounds):
    for base_station in base_stations:
      for user in base_station.users:
        #updating base stations (and users) with perturbed tracking data
        base_station.update_user_data(user.id, partitions[user.id][0: e_row])

        user.train_model()

    #Aggregating models and checking accuracy
    server.update_global_model(base_stations)

    e_row += num_user_regs

    gather_accuracy_per_round(df, server)

    if(server.can_stop_federation()):
      print("Round {}".format(round))
      save_accuracy_per_round(df)
      getting_predictions(server, base_stations)
      prepare_predictions_for_save(server)
      break